In [1]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [2]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [3]:
"""BGSA"""
def Bmove(x,a,v):
    n,dim=len(x),len(x[0])#size(x)
    v=[[random.random()*v[j][i]+a[i] for i in range(dim)] for j in range(n)]
    s=[[abs(math.tanh(_v)) for _v in vv ] for vv in v]
    temp=[[1 if rr<ss else 0 for rr,ss in zip(_r,_s)] for _r,_s in zip([[random.random() for i in range(dim)] for j in range(n)],s)]
    x_moving=[[0 if temp[ind][i]==1 else 1  for i in range(len(temp[ind])) ] for ind in range(len(temp))]
    #xm(moving)=~xm(moving)
    return x_moving,v

def mc(fit,min_f):
    fmax=max(fit)
    fmin=min(fit)
    fmean=np.mean(fit)
    i,n=1,len(fit)

    if fmax==fmin:
        m=[1 for i in range(n)]#once(n,1)
    else:
        if min_f==1:
            best=fmin
            worst=fmax
        else:
            best=fmax
            worst=fmin
        m=[(f-worst)/(best-worst) for f in fit]
    mm=[_m/sum(m) for _m in m]
    return mm

def BGc(itertion,max_iter):
    g0=1
    g=g0*(1-(itertion/max_iter))
    return g

def BGf(m,x,G,Rp,EC,itertion,max_iter):
    n,dim=len(x),len(x[0])#size(x)#n=群数,dim=次元数
    final_per=2#In the last iteration, only 2 percent of agents apply force to the others
    if EC == 1:
        kbest=final_per+(1-itertion/max_iter)*(100-final_per)
        kbest=round(n*kbest/100)
    else:
        kbest=n
    mm=np.array(m)
    am=[np.argsort(mm)[::-1][i] for i in range(len(mm))]#:
    ds=sorted(am,reverse=True)#降順

    for i in range(n):
        E=[0 for i in range(dim)]#zero(1,dim)
        for ii in range(kbest):
            j=ds[ii]
            if j != i:
                R=sum([1 for xi,xj in zip(x[i],x[j]) if xi!=xj])#hammimng dist
                R=R/dim
                for k in range(dim):
                    E[k]=E[k]+random.random()* m[j] *( (x[j][k]-x[i][k]) / (R**Rp+1/dim) )
            else:
                pass
    a=[e*G for e in E]
    return a

def BGSA(Eval_Func,n=20,m_i=200,dim=None,minf=0,prog=False,EC=1,Rp=1,f_ind=25,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            EC: Elite Check, default=1
            Rp: Value between mass, default=1
            f_ind: Value of kbest, default=25
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)

    best_bin='0'*dim
    fbest=float("-inf") if minf == 0 else float("inf")
    best_val=float("-inf") if minf == 0 else float("inf")
    #EC=1
    #Rp=1
    #f_ind=25#24: max-ones, 25: royal-road(王道)
    #minf=minf#0#1:mini,0:maximization
    gens_dict={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    #flag=dr#False
    gens=random_search(n,dim)#[[random.choice([0,1]) for _ in range(dim)] for i in range(n)]
    bestc=[]
    meanc=[]
    v=[[0 for d in range(dim)] for i in range(n)]
    fit=[float("-inf") if minf == 0 else float("inf") for i in range(n)]
    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)

    for it in miter:#
        #
        if mp!=None:
            fit=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in gens]
            alter_gens=[i for i,g in enumerate(gens) if tuple(g) not in gens_dict]
            with Pool(mp) as p:
                alter_fit = p.map(estimate,[gens[i] for i in alter_gens])
            z=0
            for i in range(len(fit)):
                if i in alter_gens:
                    gens_dict[tuple(gens[i])]=alter_fit[z]
                    fit[i]=alter_fit[z]
                    z+=1
        #
        else:
            for g_i in range(n):
                if  tuple(gens[g_i]) in gens_dict:
                    fit[g_i]=gens_dict[tuple(gens[g_i])]
                else:
                    fit[g_i]=estimate(gens[g_i])
                    gens_dict[tuple(gens[g_i])]=fit[g_i]

        if it > 1:
            if minf==1:
                pass
                #afit=find(fitness>fitold)#minimazation#find is return index_list
                afit=[ind for ind in range(n) if fit[ind] > fitold[ind]]
            else:
                #afit=find(fittness<fitold)#max#
                afit=[ind for ind in range(n) if fit[ind] < fitold[ind]]

            if len(afit)!=0:
                for ind in afit:
                    gens[ind]=gensold[ind]
                    fit[ind]=fitold[ind]

        if minf == 1:
            best=min(fit)#min
            best_ind=fit.index(min(fit))
        else:
            best=max(fit)#max
            best_ind=fit.index(max(fit))
        if it==1:
            fbest=best
            lbest=gens[best_ind]

        if minf==1:
            if best<fbest:
                fbest=dc(best)
                lbest=dc(gens[best_ind])
        else:
            if best>fbest:
                fbest=dc(best)
                lbest=dc(gens[best_ind])

        bestc=fbest
        meanc=np.mean(fit)

        m=mc(fit,minf)
        g=BGc(it,m_i)
        a=BGf(m,gens,g,Rp,EC,it,m_i)

        gensold=dc(gens)
        fitold=dc(fit)

        gens,v=Bmove(gens,a,v)
    print(len(gens_dict))
    return fbest,lbest,lbest.count(1)


In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("VoxForge512.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

900 201


In [5]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [6]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [15]:
%%time
s,g,l=BGSA(Eval_Func=Evaluate, n=1, m_i=1)

2
CPU times: user 66.9 ms, sys: 7.18 ms, total: 74.1 ms
Wall time: 249 ms


In [16]:
l

176

In [17]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,167,168,169,170,171,172,173,174,175,label
0,0.003289,0.003680,0.002809,0.002662,0.002827,0.003032,0.002014,0.004380,0.001103,0.002665,...,6.368087,15.037514,14.887147,17.031933,11.462402,0.016049,0.031459,0.009924,0.017622,English
1,0.002735,0.002047,0.002489,0.002068,0.002175,0.001964,0.002427,0.001584,0.000306,0.002996,...,8.133885,19.207247,15.687752,17.947880,11.421647,0.014899,0.029206,0.014316,0.024158,English
2,0.001556,0.001723,0.001227,0.001315,0.001594,0.001354,0.001637,0.001206,0.000508,0.001195,...,13.792500,32.569420,17.805973,20.371273,11.088083,0.014856,0.029121,0.015950,0.028974,English
3,0.007180,0.010772,0.011593,0.007111,0.014183,0.011722,0.009923,0.008713,0.002654,0.008530,...,4.076348,9.625833,1.745809,1.997327,9.626831,0.011871,0.023270,0.010955,0.016978,English
4,0.003632,0.004742,0.003760,0.002512,0.004510,0.004911,0.003478,0.004527,0.002201,0.003138,...,4.520422,10.674464,5.230475,5.984027,9.285885,0.011261,0.022074,0.008553,0.017641,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000754,0.000790,0.000955,0.000608,0.001347,0.001097,0.000515,0.000559,0.000482,0.000726,...,0.001004,0.002370,0.001476,0.001688,29.929813,0.032924,0.064538,0.026434,0.011313,Spanish
896,0.000093,0.000087,0.000061,0.000111,0.000110,0.000120,0.000210,0.000161,0.000117,0.000092,...,0.002488,0.005875,0.001386,0.001586,9.986907,0.005506,0.010794,0.003441,0.002254,Spanish
897,0.000122,0.000108,0.000083,0.000059,0.000091,0.000099,0.000052,0.000040,0.000025,0.000042,...,0.000358,0.000845,0.000601,0.000688,9.939436,0.006557,0.012853,0.005301,0.003221,Spanish
898,0.000005,0.000010,0.000008,0.000005,0.000006,0.000007,0.000008,0.000005,0.000009,0.000008,...,0.000236,0.000557,0.000454,0.000519,9.965343,0.004320,0.008469,0.003344,0.002046,Spanish


In [18]:
df_train1.to_csv('VF_BGSA.csv')